# Q2: Data Cleaning

**Phase 3:** Data Cleaning & Preprocessing  
**Points: 9 points**

**Focus:** Handle missing data, outliers, validate data types, remove duplicates.

**Lecture Reference:** Lecture 11, Notebook 1 ([`11/demo/01_setup_exploration_cleaning.ipynb`](https://github.com/christopherseaman/datasci_217/blob/main/11/demo/01_setup_exploration_cleaning.ipynb)), Phase 3. Also see Lecture 05 (data cleaning).

---

## Setup

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load data from Q1 (or directly from source)
df = pd.read_csv('data/beach_sensors.csv')
# If you saved cleaned data from Q1, you can load it:
# df = pd.read_csv('output/q1_exploration.csv')  # This won't work - load original

C:\Users\matee\AppData\Local\Temp\ipykernel_33344\1257711790.py:9: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/beach_sensors.csv')


In [2]:
print(df.columns.tolist())


['x"Station Name"', 'Measurement Timestamp', 'Air Temperature', 'Wet Bulb Temperature', 'Humidity', 'Rain Intensity', 'Interval Rain', 'Total Rain', 'Precipitation Type', 'Wind Direction', 'Wind Speed', 'Maximum Wind Speed', 'Barometric Pressure', 'Solar Radiation', 'Heading', 'Battery Life', 'Measurement Timestamp Label', 'Measurement ID']


In [7]:
df = pd.read_csv("data/beach_sensors.csv", low_memory=False)

# Strip whitespace from column names (good habit)
df.columns = df.columns.str.strip()

# Rename 
df = df.rename(columns={
    "Measurement Timestamp": "measurement_timestamp"
})


In [9]:
# --- Q2: Data Cleaning (robust version) ---

import pandas as pd
import numpy as np
import os

os.makedirs("output", exist_ok=True)

# Load dataset
df = pd.read_csv("data/beach_sensors.csv")
rows_before = len(df)

# --- Identify datetime column automatically ---
datetime_col = None
for candidate in df.columns:
    if "timestamp" in candidate.lower() or "date" in candidate.lower() or "time" in candidate.lower():
        datetime_col = candidate
        break

if datetime_col:
    print(f"Detected datetime column: '{datetime_col}'")
else:
    print("No obvious datetime column found. Skipping sort by time.")

# --- Handle Missing Data ---
missing_summary = df.isna().sum()
missing_percent = (missing_summary / len(df) * 100).round(2)

# Sort by datetime if found
if datetime_col:
    df = df.sort_values(datetime_col)

# Handle missing values with forward-fill (for time series)
for col in ['water_temp', 'air_temp']:
    if col in df.columns:
        df[col] = df[col].fillna(method='ffill')
        # Fill any remaining NaNs with median
        df[col] = df[col].fillna(df[col].median())

# --- Outlier Handling ---
outlier_info = []
for col in ['water_temp', 'air_temp']:
    if col in df.columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower, upper = Q1 - 3 * IQR, Q3 + 3 * IQR
        outliers = ((df[col] < lower) | (df[col] > upper)).sum()
        df[col] = df[col].clip(lower, upper)
        outlier_info.append((col, outliers, lower, upper))

# --- Remove duplicates ---
duplicates_removed = df.duplicated().sum()
df = df.drop_duplicates()

# --- Convert datetime column ---
if datetime_col:
    df[datetime_col] = pd.to_datetime(df[datetime_col], errors='coerce')

rows_after = len(df)

# --- Save cleaned data ---
df.to_csv("output/q2_cleaned_data.csv", index=False)
with open("output/q2_rows_cleaned.txt", "w") as f:
    f.write(str(rows_after))

# --- Create Cleaning Report ---
report_lines = []
report_lines.append("DATA CLEANING REPORT")
report_lines.append("====================\n")
report_lines.append(f"Rows before cleaning: {rows_before}\n")

report_lines.append("Missing Data Handling:")
for col in ['water_temp', 'air_temp']:
    if col in missing_summary.index:
        report_lines.append(f"- {col.replace('_', ' ').title()}: {missing_summary[col]} missing values ({missing_percent[col]}%)")
        if col == 'water_temp':
            method = "Forward-fill (time series appropriate)"
        elif col == 'air_temp':
            method = "Forward-fill, then median imputation for remaining"
        else:
            method = "N/A"
        report_lines.append(f"  Method: {method}")
        report_lines.append("  Result: All missing values filled\n")

report_lines.append("Outlier Handling:")
for col, outliers, lower, upper in outlier_info:
    report_lines.append(f"- {col.replace('_', ' ').title()}: Detected {outliers} outliers using IQR method (3×IQR)")
    report_lines.append(f"  Method: Capped at bounds [Q1 - 3×IQR, Q3 + 3×IQR]")
    report_lines.append(f"  Bounds: [{lower:.1f}, {upper:.1f}]")
    report_lines.append(f"  Result: {outliers} values capped\n")

report_lines.append(f"Duplicates Removed: {duplicates_removed}\n")

if datetime_col:
    report_lines.append("Data Type Conversions:")
    report_lines.append(f"- {datetime_col}: Converted to datetime64[ns]\n")

report_lines.append(f"Rows after cleaning: {rows_after}")

# --- Write report ---
with open("output/q2_cleaning_report.txt", "w") as f:
    f.write("\n".join(report_lines))

print("Q2 complete: Data cleaned and report saved to output/q2_cleaning_report.txt")


C:\Users\matee\AppData\Local\Temp\ipykernel_33344\3720656605.py:10: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/beach_sensors.csv")


Detected datetime column: 'Measurement Timestamp'
Q2 complete: Data cleaned and report saved to output/q2_cleaning_report.txt


---

## Objective

Clean the dataset by handling missing data, outliers, validating data types, and removing duplicates.

**Time Series Note:** For time series data, forward-fill (`ffill()`) is often appropriate for missing values since sensor readings are continuous. However, you may choose other strategies based on your analysis.

---

## Required Artifacts

You must create exactly these 3 files in the `output/` directory:

### 1. `output/q2_cleaned_data.csv`
**Format:** CSV file
**Content:** Cleaned dataset with same structure as original (same columns)
**Requirements:**
- Same columns as original dataset
- Missing values handled (filled, dropped, or imputed)
- Outliers handled (removed, capped, or transformed)
- Data types validated and converted
- Duplicates removed
- **Sanity check:** Dataset should retain most rows after cleaning (at least 1,000 rows). If you're removing more than 50% of data, reconsider your strategy—imputation is usually preferable to dropping rows for this dataset.
- **No index column** (save with `index=False`)

### 2. `output/q2_cleaning_report.txt`
**Format:** Plain text file
**Content:** Detailed report of cleaning operations
**Required information:**
- Rows before cleaning: [number]
- Missing data handling method: [description]
  - Which columns had missing data
  - Method used (drop, forward-fill, impute, etc.)
  - Number of values handled
- Outlier handling: [description]
  - Detection method (IQR, z-scores, domain knowledge)
  - Which columns had outliers
  - Method used (remove, cap, transform)
  - Number of outliers handled
- Duplicates removed: [number]
- Data type conversions: [list any conversions]
- Rows after cleaning: [number]

**Example format:**
```
DATA CLEANING REPORT
====================

Rows before cleaning: 50000

Missing Data Handling:
- Water Temperature: 2500 missing values (5.0%)
  Method: Forward-fill (time series appropriate)
  Result: All missing values filled
  
- Air Temperature: 1500 missing values (3.0%)
  Method: Forward-fill, then median imputation for remaining
  Result: All missing values filled

Outlier Handling:
- Water Temperature: Detected 500 outliers using IQR method (3×IQR)
  Method: Capped at bounds [Q1 - 3×IQR, Q3 + 3×IQR]
  Bounds: [-5.2, 35.8]
  Result: 500 values capped

Duplicates Removed: 0

Data Type Conversions:
- Measurement Timestamp: Converted to datetime64[ns]

Rows after cleaning: 50000
```

### 3. `output/q2_rows_cleaned.txt`
**Format:** Plain text file
**Content:** Single integer number (total rows after cleaning)
**Requirements:**
- Only the number, no text, no labels
- No whitespace before or after
- Example: `50000`

---

## Requirements Checklist

- [ ] Missing data handling strategy chosen and implemented
- [ ] Outliers detected and handled (IQR method, z-scores, or domain knowledge)
- [ ] Data types validated and converted
- [ ] Duplicates identified and removed
- [ ] Cleaning decisions documented in report
- [ ] All 3 required artifacts saved with exact filenames

---

## Your Approach

1. **Handle missing data** - Choose appropriate strategy (drop, forward-fill, impute) based on data characteristics
2. **Detect and handle outliers** - Use IQR method or z-scores; decide whether to remove, cap, or transform
3. **Validate data types** - Ensure numeric and datetime columns are properly typed
4. **Remove duplicates**
5. **Document and save** - Write detailed cleaning report explaining your decisions

---

## Decision Points

- **Missing data:** Should you drop rows, impute values, or forward-fill? Consider: How much data is missing? Is it random or systematic? For time series, forward-fill is often appropriate.
- **Outliers:** Are they errors or valid extreme values? Use IQR method or z-scores to detect, then decide: remove, cap, or transform. Document your reasoning.
- **Data types:** Are numeric columns actually numeric? Are datetime columns properly formatted? Convert as needed.

---

## Checkpoint

After Q2, you should have:
- [ ] Missing data handled
- [ ] Outliers addressed
- [ ] Data types validated
- [ ] Duplicates removed
- [ ] All 3 artifacts saved: `q2_cleaned_data.csv`, `q2_cleaning_report.txt`, `q2_rows_cleaned.txt`

---

**Next:** Continue to `q3_data_wrangling.md` for Data Wrangling.
